In [4]:
import os, sys

notebook_dir = os.getcwd()

#add path
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..')))

In [5]:
from scripts.correlation import *
from scripts.load_data import *
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
# Load datas
raw_analyst_data = load_data("../../Data/raw_analyst_ratings.csv")
AAPL_data = load_data("../../Data/yfinance_data/AAPL_historical_data.csv")
AMZN_data = load_data("../../Data/yfinance_data/AMZN_historical_data.csv")
GOOG_data = load_data("../../Data/yfinance_data/GOOG_historical_data.csv")
META_data = load_data("../../Data/yfinance_data/META_historical_data.csv")
MSFT_data = load_data("../../Data/yfinance_data/MSFT_historical_data.csv")
NVDA_data = load_data("../../Data/yfinance_data/NVDA_historical_data.csv")
TSLA_data = load_data("../../Data/yfinance_data/TSLA_historical_data.csv")

Data loaded successfully!
Data loaded successfully!
Data loaded successfully!
Data loaded successfully!
Data loaded successfully!
Data loaded successfully!
Data loaded successfully!
Data loaded successfully!


In [23]:
import pandas as pd

# Drop rows with NaT values in the date column
raw_analyst_data = raw_analyst_data.dropna(subset=['date'])
raw_analyst_data

,Unnamed: 0,headline,url,publisher,date,stock
0,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54,A
1,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 10:45:20,A
2,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 04:30:07,A
3,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 12:45:06,A
4,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 11:38:59,A
...,...,...,...,...,...,...
1407266,1413787,Chinese Nano-Cap Momentum Stocks Sharply Highe...,https://www.benzinga.com/movers/18/01/10994518...,Paul Quintaro,2018-01-05 11:47:36,ZX
1407267,1413788,28 Stocks Moving In Wednesday's Pre-Market Ses...,https://www.benzinga.com/news/17/12/10878295/2...,Lisa Levin,2017-12-06 09:12:01,ZX
1407268,1413789,China Zenix Auto International Reports Q3 EPAD...,https://www.benzinga.com/news/earnings/17/12/1...,Paul Quintaro,2017-12-06 07:04:31,ZX
1407269,1413790,46 Biggest Movers From Yesterday,https://www.benzinga.com/news/17/11/10788120/4...,Lisa Levin,2017-11-15 06:04:52,ZX


In [ ]:
# Attempt to convert dates again
raw_analyst_data['date'] = pd.to_datetime(raw_analyst_data['date'], errors='coerce')
AMZN_data['Date'] = pd.to_datetime(AMZN_data['Date'], errors='coerce')

# Check for any NaT values after conversion
if raw_analyst_data['date'].isnull().any():
    print("There are still unparseable dates in raw_analyst_data.")
else:
    print("All dates in raw_analyst_data are parseable.")

if AMZN_data['Date'].isnull().any():
    print("There are unparseable dates in AMZN_data.")
else:
    print("All dates in AMZN_data are parseable.")

# Merge the data
merged_data = pd.merge(raw_analyst_data, AMZN_data, left_on='date', right_on='Date', how='inner')

# Sentiment analysis
merged_data['sentiment'] = merged_data['headline'].apply(analyze_sentiment_vader)

# Calculate daily stock returns
merged_data['daily_return'] = merged_data['Close'].pct_change()

# Display the merged data
print(merged_data)

All dates in raw_analyst_data are parseable.
All dates in AMZN_data are parseable.


TypeError: 'float' object is not iterable

In [27]:
merged_data

,Unnamed: 0,headline,url,publisher,date,stock,Date,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits,sentiment,daily_return


In [ ]:
# Correlation analysis
correlation = merged_data['sentiment'].corr(merged_data['daily_return'])
print(f"Correlation between sentiment and stock returns: {correlation}")

# Granger causality test
granger_test = granger_causality_test(merged_data['sentiment'], merged_data['daily_return'])
print("Granger Causality Test Results:")
for lag, result in granger_test.items():
    print(f"Lag {lag}: p-value = {result[0]['ssr_ftest'][1]}")

# Heatmap of correlations
correlation_matrix = merged_data[['sentiment', 'daily_return']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title("Correlation Heatmap")
plt.show()